# 2. Modelling SVR Linear

---

In [ ]:
## load modules and run mlflow_logging.ipynb to get function to track model information on MLFLow
import sys
sys.path.append("..")
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import LinearSVR, SVR
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from datetime import datetime
import pickle
import os


from sklearn.model_selection import GridSearchCV

from modeling.functions import modelling, log_to_mlflow, get_features, save_models, load_models

In [ ]:
data = pd.read_csv('../data/GEFCom2014Data/Wind/raw_data_incl_features.csv', parse_dates=['TIMESTAMP'])
data.head()
data.dropna(inplace=True)
data.info()
RSEED = 42

In [ ]:
data = pd.get_dummies(data, columns = ['WD100CARD','WD10CARD'])
data.head()

In [ ]:
## train-test-split
data_train, data_test = train_test_split(data, test_size=0.25, random_state=RSEED, stratify=data.ZONEID)

In [ ]:
# define features and feature dict
feature_dict = get_features(data)

# features = feature_dict['all']

In [ ]:
# # define zone
# zone = 1

In [ ]:
# # split train and test data in feature and TARGETVAR parts and cut data to desired zones
# X_train = data_train[data_train.ZONEID == zone][features]
# y_train = data_train[data_train.ZONEID == zone].TARGETVAR

# X_test = data_test[data_test.ZONEID == zone][features]
# y_test = data_test[data_test.ZONEID == zone].TARGETVAR

In [ ]:
# # Scale data
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

In [ ]:
# model = SVR()
# for ii in range(5, 6):
#     model = SVR(kernel='poly', degree=ii, C=1, cache_size=100)
#     model.fit(X_train, y_train)
#     y_pred_train = model.predict(X_train)
#     y_pred_train = [1 if value >= 1 else 0 if value <= 0 else value for value in y_pred_train]
#     y_pred = model.predict(X_test)
#     y_pred = [1 if value >= 1 else 0 if value <= 0 else value for value in y_pred]
#     print(ii,': train',mean_squared_error(y_train, y_pred_train, squared=False),
#             ', test:', mean_squared_error(y_test, y_pred, squared=False))

In [ ]:

# model = SVR(kernel='rbf', gamma='auto', C=10)
# model.fit(X_train, y_train)
# y_pred_train = model.predict(X_train)
# y_pred_train = [1 if value >= 1 else 0 if value <= 0 else value for value in y_pred_train]
# y_pred = model.predict(X_test)
# y_pred = [1 if value >= 1 else 0 if value <= 0 else value for value in y_pred]
# print('train',mean_squared_error(y_train, y_pred_train, squared=False),
#         ', test:', mean_squared_error(y_test, y_pred, squared=False))

In [ ]:
model = SVR()
scaler = MinMaxScaler()

model_dict = {}
results = {}
results_train = {}
param_grid = [ 
    # {           'C': [0.1, 1, 5, 10], 
    #             'degree': [5, 6, 7],
    #             'kernel': ['poly']
    #             },
    {           'C': [0.1, 1, 5, 10, 100], 
                'kernel': ['rbf']
                }
]


for key in feature_dict.keys():
    print(f'Features: {key}')
    results_train[key],results[key], model_dict[key] = modelling(data_train, data_test, feature_dict[key], 
                                                                    model = model, 
                                                                    scaler = scaler, 
                                                                    print_scores = True, 
                                                                    log = True, 
                                                                    infotext_mlflow = None, 
                                                                    save_model = True, 
                                                                    perform_gridCV = True, 
                                                                    param_grid = param_grid, 
                                                                    n_jobs = 3)
    results[key] = {k : np.round(value,5) for k,value in results[key].items()}

In [ ]:
model_dict